# Nettoyage du fichier "olympic_result.csv"

In [1]:
from pyspark.sql import SparkSession
import pandas as pd
from IPython.display import display

# Create a Spark session
spark = SparkSession.builder \
    .appName("Olympic Results Cleaning") \
    .getOrCreate()

# Path to your CSV file
csv_file_path = "olympic_results.csv"

# Read the CSV file into a DataFrame
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

# Show the schema of the DataFrame
df.printSchema()

# Convert the first 5 rows to a Pandas DataFrame for a pretty display
pandas_df = df.limit(5).toPandas()

# Display the Pandas DataFrame
display(pandas_df)






root
 |-- Unnamed: 0: integer (nullable = true)
 |-- discipline_title: string (nullable = true)
 |-- event_title: string (nullable = true)
 |-- slug_game: string (nullable = true)
 |-- participant_type: string (nullable = true)
 |-- medal_type: string (nullable = true)
 |-- athletes: string (nullable = true)
 |-- rank_equal: string (nullable = true)
 |-- rank_position: string (nullable = true)
 |-- country_name: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- country_3_letter_code: string (nullable = true)
 |-- athlete_url: string (nullable = true)
 |-- athlete_full_name: string (nullable = true)
 |-- value_unit: string (nullable = true)
 |-- value_type: string (nullable = true)



,Unnamed: 0,discipline_title,event_title,slug_game,participant_type,medal_type,athletes,rank_equal,rank_position,country_name,country_code,country_3_letter_code,athlete_url,athlete_full_name,value_unit,value_type
0,0,Curling,Mixed Doubles,beijing-2022,GameTeam,GOLD,"[('Stefania CONSTANTINI', 'https://olympics.co...",False,1,Italy,IT,ITA,None,None,None,None
1,1,Curling,Mixed Doubles,beijing-2022,GameTeam,SILVER,"[('Kristin SKASLIEN', 'https://olympics.com/en...",False,2,Norway,NO,NOR,None,None,None,None
2,2,Curling,Mixed Doubles,beijing-2022,GameTeam,BRONZE,"[('Almida DE VAL', 'https://olympics.com/en/at...",False,3,Sweden,SE,SWE,None,None,None,None
3,3,Curling,Mixed Doubles,beijing-2022,GameTeam,None,"[('Jennifer DODDS', 'https://olympics.com/en/a...",False,4,Great Britain,GB,GBR,None,None,None,None
4,4,Curling,Mixed Doubles,beijing-2022,GameTeam,None,"[('Rachel HOMAN', 'https://olympics.com/en/ath...",False,5,Canada,CA,CAN,None,None,None,None


In [2]:
df = df.drop('Unnamed: 0')

In [3]:
#Remplir les valeurs manquantes

df_cleaned = df.fillna({
    'athletes': 'Unknown',
    'rank_equal': False,
    'rank_position': -1,
    'athlete_url': 'Unknown',
    'athlete_full_name': 'Unknown',
    'value_unit': 'Unknown',
    'value_type': 'Unknown'
})

In [4]:
# Suppression des lignes avec des valeurs manquantes sur des colonnes importante 
df_cleaned = df_cleaned.dropna(how='any', subset=['discipline_title', 'event_title', 'slug_game', 'country_name', 'medal_type'])

# DSupression des doublons 
df_cleaned = df_cleaned.dropDuplicates()

# Display the cleaned DataFrame
pandas_df_cleaned = df_cleaned.limit(10).toPandas()
display(pandas_df_cleaned)

,discipline_title,event_title,slug_game,participant_type,medal_type,athletes,rank_equal,rank_position,country_name,country_code,country_3_letter_code,athlete_url,athlete_full_name,value_unit,value_type
0,Alpine Skiing,Men's Downhill,beijing-2022,Athlete,GOLD,Unknown,False,1,Switzerland,CH,SUI,https://olympics.com/en/athletes/beat-feuz,Beat FEUZ,1:42.69,TIME
1,Cross Country Skiing,Men's 50km Mass Start Free,beijing-2022,Athlete,SILVER,Unknown,False,2,ROC,ROC,ROC,https://olympics.com/en/athletes/ivan-yakimushkin,Ivan YAKIMUSHKIN,1:11:38.2,TIME
2,Cross Country Skiing,Men's Sprint Free,beijing-2022,Athlete,GOLD,Unknown,False,1,Norway,NO,NOR,https://olympics.com/en/athletes/klaebo,Johannes Hoesflot KLAEBO,Unknown,Unknown
3,Diving,Men's Synchronised 10m Platform,tokyo-2020,GameTeam,BRONZE,"[('Aleksandr BONDAR', 'https://olympics.com/en...",False,3,ROC,ROC,ROC,Unknown,Unknown,439.92,POINTS
4,Boxing,Men's Feather (52-57kg),tokyo-2020,Athlete,BRONZE,Unknown,True,3,Cuba,CU,CUB,https://olympics.com/en/athletes/lazaro-alvare...,Lazaro ALVAREZ ESTRADA,Unknown,Unknown
5,Athletics,Women's Triple Jump,tokyo-2020,Athlete,BRONZE,Unknown,False,3,Spain,ES,ESP,https://olympics.com/en/athletes/ana-peleteiro,Ana PELETEIRO,14.62,DISTANCE
6,Shooting,double trap 150 targets men,rio-2016,Athlete,GOLD,Unknown,false,1,Independent Olympic Athletes,IOA,IOA,https://olympics.com/en/athletes/fehaid-aldeehani,Fehaid ALDEEHANI,Unknown,Unknown
7,Cycling Track,Omnium men,rio-2016,Athlete,BRONZE,Unknown,false,3,Denmark,DK,DEN,https://olympics.com/en/athletes/lasse-norman-...,Lasse Norman HANSEN,192,POINTS
8,Athletics,100m women,rio-2016,Athlete,BRONZE,Unknown,false,3,Jamaica,JM,JAM,https://olympics.com/en/athletes/shelly-ann-fr...,Shelly-Ann FRASER-PRYCE,10860,TIME
9,Canoe Sprint,K-1 200m (kayak single) women,rio-2016,Athlete,GOLD,Unknown,false,1,New Zealand,NZ,NZL,https://olympics.com/en/athletes/lisa-carrington,Lisa CARRINGTON,Unknown,Unknown
